<a href="https://colab.research.google.com/github/CoderAnubhav/Gideon-AI-Chatbot/blob/master/Gideon(QA)_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 37.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Fou

In [2]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=7fc460fd425c7087127cf5eb0056ec570c6c3b4f77a91a78a40fc23c9a82dfaf
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [3]:
from bs4 import BeautifulSoup
from googlesearch import search
import joblib
from urllib.request import urlopen
from lxml import  html
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import random
import string
import requests
import wikipedia as wk
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)      # for  removing all form of output truncations
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',None)

# dataset coronavirus WHO
url='https://raw.githubusercontent.com/CoderAnubhav/Gideon-AI-Chatbot/master/WHO_FAQ1.xlsx'      #link for the dataset
response_encodings_url='https://raw.githubusercontent.com/CoderAnubhav/Gideon-AI-Chatbot/master/enc_data.pkl'  #link for the response encodings on the dataset given by the above url using Universal Sentence Encoder multilingual Qa
data=pd.read_excel(url,index_col=False)

data.head()

,Context,Answer,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,What is a coronavirus?,Coronaviruses are a large family of viruses which may cause illness in animals or humans.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,What is a coronavirus?,"In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,What is COVID-19?,"COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,What are the symptoms of COVID-19?,"The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have aches and pains, nasal congestion, runny nose, sore throat or diarrhea. These symptoms are usually mild and begin gradually.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,What are the symptoms of COVID-19?,Some people become infected but don’t develop any symptoms and don't feel unwell. Most people (about 80%) recover from the disease without needing special treatment.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6','Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23'],axis=1,inplace=True)

In [5]:
data.head()

,Context,Answer
0,What is a coronavirus?,Coronaviruses are a large family of viruses which may cause illness in animals or humans.
1,What is a coronavirus?,"In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases"
2,What is COVID-19?,"COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019."
3,What are the symptoms of COVID-19?,"The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have aches and pains, nasal congestion, runny nose, sore throat or diarrhea. These symptoms are usually mild and begin gradually."
4,What are the symptoms of COVID-19?,Some people become infected but don’t develop any symptoms and don't feel unwell. Most people (about 80%) recover from the disease without needing special treatment.


In [9]:
# Paper link for USE -> https://openreview.net/forum?id=rJvJXZb0W
# Load module containing USE
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

In [ ]:
# Refernec for RegEX -> https://www.w3schools.com/python/python_regex.asp

def preprocess_sentences(input_sentences):
    return [re.sub(r'(covid-19|covid)', 'coronavirus', input_sentence, flags=re.I)
            for input_sentence in input_sentences]

def generateResponseUtil(input_ques):

      '''
      response_encodings = module.signatures['response_encoder'](
        input=tf.constant(data['Answer']),
        context=tf.constant(data['Context']))['outputs']
      '''

      response_encodings=joblib.load(urlopen(response_encodings_url))       # Loading saved response encodings instead of  creating them every time since it remains same

      question_encodings = module.signatures['question_encoder'](
          tf.constant(preprocess_sentences(input_ques))
      )['outputs']

      # Get the responses
      test_responses = data.Answer[np.argmax(np.inner(question_encodings, response_encodings), axis=1)]   # Finding the best response for the given question using inner product between the question and the response encodings

      # Show them in a dataframe
      df=pd.DataFrame({'Test Questions': input_ques, 'Test Responses': test_responses})

      return df['Test Responses'].to_string(index=False)

welcome_input = ("hello gideon","hi gideon","hey gideon")
welcome_response = ["hi", "hey", "*nods*", "hi there", "hello"]
def welcome(user_response):
    if user_response.lower() in welcome_input:
          return random.choice(welcome_response)


def generateResponse(user_ques):
    robo_response=''
    if user_ques != "" :
      user_ques=[user_ques]
      user_ques=preprocess_sentences(user_ques)
      robo_response+=generateResponseUtil(user_ques)

    print("Gideon : "+robo_response)

#wikipedia search
def wikipedia_query(input):
    reg_ex = re.search('wiki (.*)', input)          # capture and group everything after wiki
    #print(reg_ex)

    try:
        if reg_ex:
            topic = reg_ex.group(1)                 # group after wiki(0)
            #print(topic)
            wiki = wk.summary(topic, sentences = 3)
            return wiki
    except Exception as e:
            return 0

#google search
def google_query(query,index=0):
    fallback="Gideon : Sorry,I cannot find a response in google"
    reg_ex = re.search('google (.*)', query)                        # capture and group everything after google


    query=reg_ex.group(1)                                           # group after google (0)

    try:
      search_result_list=list(search(query,tld="co.in",num=10,stop=3,pause=1))       # A list that stores the URLs of the search results from Google.
      page=requests.get(search_result_list[index])                                   # Retrieves the content of a specific search result page using the requests library
      soup=BeautifulSoup(page.content,features="lxml")                               # Parses the HTML content of the page using BeautifulSoup with the lxml parser.
      article_text=''
      article=soup.findAll('p')                                                      # Finds all 'p' elements on the page using BeautifulSoup.
      for element in article:
        article_text+='\n'+''.join(element.findAll(text=True))                       # The loop extracts the text content from each 'p' element and appends it to article_text.
      print("Gideon : Showing response from Google...")
      print(article_text)


    except:
      print(fallback)

def chatbot_query():
    flag=True
    print("***************************************************")
    print("My name is Gideon and I'm a COVID-19 assistance chatbot to resolve your general queries about COVID-19 in multiple languages.")
    print("I sometime give illogical answers. If you are unsatisfied with my response you can ask me to get responses from google or wikipedia by typing google (or wiki) followed by your query")
    print("You can start a conversation with me by typing Hello Gideon or Hi Gideon")
    print("If you want to exit,type bye")
    while(flag==True):
        user_response = input()
        user_response=user_response.lower()
        if (user_response not in ['bye','shutdown','exit', 'quit','bye gideon']):
            if (user_response=='thanks' or user_response=='thank you' or user_response=='okay' or user_response=='ok'):
                print("Gideon : You are welcome..")
                print("Gideon : Do you want to continue?(Yes/No)")
                flag=True if input().lower()=='yes' else False

            elif "wiki" in user_response:
                w=wikipedia_query(user_response)
                print("Gideon : Showing data from Wikipedia..")
                if w==0:
                  print("No data avilable on wikipedia for the following query")
                else:
                  print(w)

            elif "google" in user_response:
                google_query(user_response)

            else:
                if(welcome(user_response)!=None):
                    print("Gideon : "+welcome(user_response))
                else:
                    generateResponse(user_response)

        else:
          flag=False

    if flag==False:
      print("Gideon : It's a pleasure talking to you.Stay Safe.Bye!!! ")

chatbot_query()